# 🏗️ Notebook: Einführung in Retrieval-Augmented Generation (RAG) [mit Langchain v1]

In diesem Notebook lernen wir, wie wir **Retrieval-Augmented Generation (RAG)** nutzen können, um die Antworten von großen Sprachmodellen (LLMs) durch externe Wissensquellen zu verbessern.

## 📚 Quellen

- [LangChain Dokumentation zu RAG](https://python.langchain.com/docs/tutorials/rag/)

## 🔄 Änderungen

Dieses Notebook wurde auf **LangChain 1.x** aktualisiert. Die Hauptänderungen sind:

- **LangChain-Version**: Aktualisiert von 0.3.27 auf 1.1.3 (und zugehörige Pakete).
- **Veraltete Komponenten entfernt**: `RetrievalQA` und `PromptTemplate` sind deprecated und wurden durch moderne Runnable-Objekte ersetzt.
- **Runnable-Pipelines**: Anstatt Chains verwendet das Notebook jetzt die LangChain Expression Language (LCEL) mit `RunnablePassthrough`, `RunnableLambda` und dem `|` Operator für die Verkettung von Retriever, Prompt und LLM.
- **Prompt-Template**: `PromptTemplate` (deprecated) wurde durch `ChatPromptTemplate` ersetzt.

Der Funktionsumfang bleibt derselbe, aber der Code folgt nun den aktuellen Best Practices von LangChain.

---

In [1]:
%%capture
# Installieren wir zunächst einige Packete die uns LangChain zur Verfügung stellt. U.a. eine Schnittstelle zu Ollama.
!uv add langchain==1.1.3 langchain-community==0.4.1 langchain_text_splitters==1.0.0 langchain-core==1.1.3 langchain-ollama==1.0.0 pypdf rank_bm25 chromadb

In [2]:
import langchain
langchain.__version__

'1.1.3'

In [3]:
# Import necessary libraries
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

In [4]:
LLM_URL = "http://132.199.138.16:11434"
LLM_MODEL = "gpt-oss:20b" 
EMBEDDING_MODEL = "nomic-embed-text" # Der Ollama-Server bei der Uni stellt uns ein Embedding Modell zur Verfügung mit dem wir dokumente in Vektoren umwandeln können.

In [5]:
# LLM initialisieren
llm = ChatOllama(
    model=LLM_MODEL,
    base_url=LLM_URL,
    temperature=0
)

## 1. Dokumente laden und vorbereiten

Wir nehmen als Beispiel ein PDF-Dokument zur Baugeschichte der Universität Regensburg.

Zunächst folgen wir drei Schritte:

* Herunterladen der PDF von der offiziellen Webseite (auch manuell möglich!)
* Wir wandeln das PDF in einen Text um
* Aufteilung in Chunks

In [6]:
path_pdf = "https://www.uni-regensburg.de/fileadmin/user_upload/universitaet/archiv/Forschungshilfen/Dokumente_Unigeschichte/Infrastruktur_und_Baugeschichte/Chronik_Baugeschichte.pdf"

# download as "ur_baugeschichte.pdf" falls not already present
import os, requests
if not os.path.exists("ur_baugeschichte.pdf"): 
    response = requests.get(path_pdf)
    with open("ur_baugeschichte.pdf", "wb") as f:
        f.write(response.content)

### Dokumente laden

In Langchain gibt es viele Möglichkeiten, Dokumente zu laden. Abhängig vom Dateityp (z.B. PDF, DOCX, TXT, HTML) gibt es verschiedene Loader-Klassen. 

In unserem Fall verwenden wir den `PyPDFLoader`, um die PDF-Datei zu laden.

Weitere Beispiele für Loader:

```python
# 1. Für PDFs
from langchain_community.document_loaders import PyPDFLoader
# 2. Für Textdateien / Strings
from langchain_community.document_loaders import TextLoader
# 3. Für Webseiten
from langchain_community.document_loaders import WebBaseLoader
# 4. Für HTML-Dateien
from langchain_community.document_loaders import HTMLLoader
```

Mehr Informationen zu den verschiedenen Loadern findet ihr auch in der [LangChain Dokumentation](https://docs.langchain.com/oss/javascript/integrations/document_loaders).

Alle Loader-Klassen haben eine `load()`-Methode, die die Datei einliest und in eine Liste von `Document`-Objekten umwandelt. 
Jedes `Document`-Objekt enthält den Textinhalt.

In [7]:
# Dokument laden
# Mehr zum Document-Objekt (https://docs.langchain.com/oss/python/integrations/document_loaders).
loader = PyPDFLoader("ur_baugeschichte.pdf")
full_pdf = loader.load()  # load() gibt eine Liste von Document-Objekten zurück, ein Document pro Seite im PDF

# Text in Chunks aufteilen
# RecursiveCharacterTextSplitter teilt lange Texte in kleinere Abschnitte (Chunks)
# Dies ist wichtig für RAG-Systeme, da Embedding-Modelle oft Längenbeschränkungen haben von z:.B. 512 oder 1024 Token
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,        # Maximale Länge eines Chunks in Zeichen
    chunk_overlap=50,      # Überlappung zwischen Chunks verhindert Informationsverlust an Grenzen
    separators=["\n", ",", " ", ""]       # Text wird primär an Absätzen (Doppel-Zeilenumbrüchen) getrennt
)
# split_documents() wendet das Splitting auf das Document-Objekt an
# und gibt eine Liste von kleineren Document-Objekten zurück
docs = text_splitter.split_documents(full_pdf)

# page_content enthält den eigentlichen Text eines Document-Objekts
print(f"Anzahl der Chunks: {len(docs)}")
print(f"\nBeispiel-Chunk:\n{docs[0].page_content}")

Anzahl der Chunks: 32

Beispiel-Chunk:
Chronik zur universitären Baugeschichte Regensburg 
 
Datum Ereignis Gebäude 
28.07.1964 Erteilung des Planungsauftrages für das 
Sammelgebäude 
 
Quelle: RUZ 3/69, S. 8 
Sammelgebäude (RWS) 
1965 Einrichtung eines Universitätsbauamtes 
 
Quelle: RUZ 1/65, S. 24 
Bauamt 
1965 Beginn der Umbaumaßnahmen am 
ehemaligen Gebäude des Albertus-
Magnus-Gymnasiums (Ägidienplatz) für 
die vorläufige Unterbringung der 
Universitätsbibliothek  
 
Quelle: RUZ 1/65, S. 23-24 
Bibliothek 
11.05.1965 Erteilung des Planungsauftrages für die 
Mensa 
 
Quelle: RUZ 3/69, S. 8 
Mensa 
20.11.1965 Grundsteinlegung des Sammelgebäudes 
 
Quelle: RUZ 9/66, S. 4 
Sammelgebäude (RWS) 
1965 Ausschreibung des ersten Wettbewerbs 
für Einzelbaumaßnahmen auf Grundlage 
der strukturellen Rahmenplanung 
 
Quelle: RUZ 10/66, S. 8 
Mensa 
28./29.01.1966 Bauwettbewerb Mensa mit Massenstudie 
für das Zentrum 
 
Erster Preis: Max Dömges 
 
Quelle: RUZ 1/66, S. 4; RUZ 10/66, S. 8; 
RUZ 1

Der `RecursiveCharacterTextSplitter` versucht die Separatoren von oben nach unten in der angegebenen Reihenfolge. Er nimmt immer den ersten Separator, der den Text in Chunks `≤` `chunk_size` aufteilen kann. Falls ein Chunk trotzdem noch zu groß ist, probiert er den nächsten Separator in der Liste, bis der Text klein genug ist.

Der `chunk_overlap` sorgt dafür, dass aufeinanderfolgende Chunks sich überlappen. Die letzten X Zeichen eines Chunks werden am Anfang des nächsten Chunks wiederholt, damit wichtige Informationen, die über Chunk-Grenzen hinausgehen, nicht verloren gehen und der semantische Kontext erhalten bleibt. [Hier](https://dev.to/tak089/what-is-chunk-size-and-chunk-overlap-1hlj)
 findet ihr eine weitere Erklärung dazu.

## 2. RAG mit Keyword-basierter Suche (BM25)

BM25 ist ein klassischer Ranking-Algorithmus, der auf **Keyword-Matching** basiert. Er berechnet, wie relevant ein Dokument für eine Suchanfrage ist, basierend auf der Häufigkeit und Verteilung der Suchbegriffe in den Chunks. BM25 berechnet einen Relevanz-Score für jedes Dokument und sortiert sie entsprechend.



In [8]:
bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = 10  # Anzahl der zurückgegebenen Dokumente

# Beispiel-Suche mit dem BM25-Retriever
# mit .invoke() werden die 10 relevantesten Dokumente zurückgegeben
query = "Wann erfolgte die Inbetriebnahme des Vielberth-Gebäudes?"
retrieved_docs = bm25_retriever.invoke(query)

### Ggf. Auskommentieren um die gefundenen Dokumente anzuzeigen: ###
# print(f"Query: {query}\n")
# print("Gefundene Dokumente:")
# for i, doc in enumerate(retrieved_docs):
#     print(f"\n--- Dokument {i+1} ---")
#     print(doc.page_content)

In [9]:
# Erstellen wir eine Prompt mit ChatPromptTemplate. ChatPromptTemplate ermöglicht es uns, flexible Prompts mit Platzhaltern zu erstellen.
# Die Platzhalter werden später durch die tatsächlichen Werte ersetzt
template = """Beantworte die Frage basierend auf dem folgenden Kontext:

Kontext: {context}

Frage: {question}

Antwort:"""

# Funktion zum Formatieren der abgerufenen Dokumente in einen String
def format_docs(docs):
    # # Alle Dokumente werden zu einem String zusammengefügt, getrennt durch zwei Zeilenumbrüche
    return "\n\n".join(doc.page_content for doc in docs)

# Die input_variables müssen mit den Platzhaltern im Template übereinstimmen
prompt = ChatPromptTemplate.from_template(template)

# Die RAG-Pipeline für BM25:
# 1. bm25_retriever: Holt die relevantesten Dokumente zur Frage basierend auf Keyword-Matching
# 2. RunnableLambda(format_docs): Wandelt die Liste von Dokumenten in einen zusammenhängenden String um
# 3. RunnablePassthrough(): Leitet die Eingabe (die Frage) direkt weiter, ohne sie zu verändern. In unserem Beispiel question_1
# 4. prompt: Setzt den formatierten Kontext und die Frage in den Prompt ein
# 5. llm: Generiert die finale Antwort basierend auf dem Prompt. Prompt erwartet "context" und "question" als Eingaben, daher das Dictionary am Anfang.
# "|" : Verkettet Runnable-Objekte in der LangChain Expression Language (LCEL), wobei die Ausgabe des linken Objekts die Eingabe des rechten wird.
qa_chain_bm25 = (
    {"context": bm25_retriever | RunnableLambda(format_docs), "question": RunnablePassthrough()} 
    | prompt
    | llm
)

# Fragen stellen
question_1 = "Wann erfolgte die Inbetriebnahme des Vielberth-Gebäudes?"
result = qa_chain_bm25.invoke(question_1) # Mit .invoke() wird die Kette ausgeführt
print(f"Frage: {question_1}")
print(f"Antwort: {result.content}")

Frage: Wann erfolgte die Inbetriebnahme des Vielberth-Gebäudes?
Antwort: Die Inbetriebnahme des Vielberth‑Gebäudes erfolgte im **Mai 2011**.


## 3. RAG mit Embeddings (Semantische Suche)

Bei der **Embedding-basierten Suche** werden Texte in numerische Vektoren umgewandelt, die ihre semantische Bedeutung repräsentieren. 

Wir verwenden ChromaDB (oft als "Chroma" bezeichnet), eine Open-Source Vektor-Datenbank. Jedem Vektor wird ein Text-Chunk zugeordnet, sodass wir bei einer Suchanfrage die semantisch ähnlichsten Chunks finden können.

Mit `vectorstore.get()` könnten wir alle Vektoren abrufen als Python-Dictionary.

In [10]:
# Embeddings-Modell initialisieren.
# Auf dem Ollama-Server der Uni ist das Modell "nomic-embed-text" verfügbar, mit dem wir Texte in Vektoren umwandeln können.
# Wichtige Hinweise: 
# (1) Die Umwandlung in Vekotren braucht Zeit und Rechenressourcen. Je nach Dokumentenlänge kann dies mehrere Minuten dauern, also bitte Geduld haben bzw. Daten gut vorbereiten!
# (2) Das Embedding-Modell hat eine Längenbeschränkung von ca. 512 Token. Längere Texte müssen vorher in Chunks aufgeteilt werden!

ollama_embeddings = OllamaEmbeddings(
    model=EMBEDDING_MODEL,
    base_url=LLM_URL
)

try:
  # Dieses Codestück habe ich hinzugefügt, um sicherzustellen, dass bei wiederholtem Ausführen des Notebooks kein doppelter Vektor-Speicher entstehen.
  vectorstore.delete_collection() 
except:
  pass

# Mit Chroma einen Vektor-Speicher aus den Dokumenten erstellen.
vectorstore = Chroma.from_documents(docs, ollama_embeddings)

# Retriever erstellen. Der Parameter k gibt an, wie viele ähnliche Dokumente zurückgegeben werden sollen.
vector_retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# Beispiel-Suche
query = "Was passierte 1980 an der Universität Regensburg?"
retrieved_docs = vector_retriever.invoke(query)

print(f"Query: {query}\n")
print("Gefundene Dokumente:")
for i, doc in enumerate(retrieved_docs[:1]):
    print(f"\n--- Dokument {i+1} ---")
    print(doc.page_content)

Query: Was passierte 1980 an der Universität Regensburg?

Gefundene Dokumente:

--- Dokument 1 ---
Quelle: RUZ 7/79, S. 2 
10.12.1979 Das Universitätsbauamt legt zwei 
unterschiedliche Ausbauvarianten der 
Mensa als Haushaltsunterlage HU-Bau 
zur Prüfung vor, die beide von einer 
baulichen Erweiterung des 
Mensagebäudes nach Westen ausgingen   
Mensa 
1980 Fertigstellung des Botanischen Versuchs- 
und Lehrgartens der Universität 
 
Quelle: RUZ 3/82, S. 7 
Botanischer Garten 
20.05.1980 Genehmigung eines 
Ergänzungsprogramms und Erteilung des 
Planungsauftrags für den Fachbereich 
Chemie Pharmazie 
Chemie Pharmazie 
22.07.1980 Richtfest der ersten Baustufe des 
Klinikums (ZMK) 
 
Quelle: RUZ 6/80, S. 2 
Klinikum 
1983 Die UR ist Schauplatz für die ARD-
Fernsehserie „Matt in 13 Zügen“, die ab 
1984 ausgestrahlt wird (v.a. die Räume 
von Prof. Dr. Märkl/Chemie u. die 
Pizzeria). (RUZ 1983) Darsteller waren 
u.a. Gudrun Landgrebe, Mathieu 
Carrière, Tommy Piper und Peer 
Augustinski. 
Chem

In [11]:
# Wir können nun eine RAG-Chain mit dem Embedding-basierten Retriever erstellen.
qa_chain_embeddings = (
    {"context": vector_retriever | RunnableLambda(format_docs), "question": RunnablePassthrough()}
    | prompt
    | llm
)

# Frage stellen
result = qa_chain_embeddings.invoke(query)
print(f"Frage: {query}")
print(f"Antwort: {result.content}")

Frage: Was passierte 1980 an der Universität Regensburg?
Antwort: Im Jahr 1980 gab es mehrere bedeutende Bau- und Planungsereignisse an der Universität Regensburg:

| Datum | Ereignis |
|-------|----------|
| **20. Mai 1980** | Genehmigung eines Ergänzungsprogramms und Erteilung des Planungsauftrags für den Fachbereich Chemie / Pharmazie. |
| **22. Juli 1980** | Richtfest der ersten Baustufe des Klinikums (Zahn‑, Mund‑ und Kieferklinik – ZMK). |
| **1980 (Gesamt)** | Fertigstellung des Botanischen Versuchs‑ und Lehrgartens der Universität. |

Kurz gesagt: 1980 wurde der Botanische Garten fertiggestellt, ein ergänzendes Programm für Chemie / Pharmazie genehmigt und ein Planungsauftrag erteilt, und die erste Baustufe des Klinikums erhielt ihr Richtfest.


## Übung: Daten aus mehreren Textdateien laden

In der Praxis möchte man oft nicht nur ein einzelnes Dokument verwenden, sondern eine ganze Sammlung von Textdateien als Wissensbasis für RAG nutzen.
Recherchiert, wie man mit `DirectoryLoader` mehrere Textdateien aus einem Verzeichnis laden kann.

Über GRIPS habe ich euch einige Textdateien zur Verfügung gestellt, die ihr für RAG verwenden könnt.
Es handelt sich um kurze Berichte zu Sonnenfinsternissen in den kommenden Jahrzehnten.

So könnt ihr eine RAG-Pipeline mit diesen Textdateien erstellen:

* 1. Ladet alle Textdateien aus `datenbank/` mit `DirectoryLoader`
* 2. Erstellt einen Vektor-Speicher mit Chroma und Ollama-Embeddings
* 3. Verwendet wieder einen `RecursiveCharacterTextSplitter`, um die Dokumente in Chunks aufzuteilen. Da unser Embedding-Modell eine maximale Eingabelänge von 512 Tokens hat, setzen wir die `chunk_size` auf 500 und `chunk_overlap` auf 50, um sicherzugehen, dass die Chunks die Längenbeschränkung einhalten.
* 4. Erstellt einen `vector_retriever`, der die 5 semantisch ähnlichsten Chunks für eine Suchanfrage zurückgibt.
* 5. Testet den `vector_retriever` mit einer Beispiel-Suchanfrage, z.B. "Wann gibt es bei den Polargebieten eine totale Sonnenfinsternis?"
* 6. Erstellt eine RAG-Chain mit dem Embedding-basierten Retriever und testet sie mit der gleichen Suchanfrage.


In [12]:
# Hier Code einfügen...

<details>
<summary><b>Lösung anzeigen</b></summary>

<p>Dokumente aus mehreren Textdateien laden:</p>
<br/>
-------

```python
from langchain_community.document_loaders import DirectoryLoader, TextLoader

loader = DirectoryLoader(
    "datenbank/",
    glob="*.txt", # Nur .txt Dateien laden
    loader_cls=TextLoader
)

documents = loader.load()

# Wichtig!: Zwar sind die Dokumente in der Datenbank meist schon kurz, aber wir teilen sie dennoch in Chunks auf, um sicherzugehen,
# dass sie die Längenbeschränkungen der Embedding-Modelle einhalten.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n", ",", " ", ""]
)

docs = text_splitter.split_documents(documents)
vectorstore.delete_collection()
vectorstore = Chroma.from_documents(docs, ollama_embeddings)
vector_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
```

<p>Testen des `vector_retriever` mit einer Beispiel-Suchanfrage:</p>
<br/>
-------

```python
# Beispiel-Suche
query = "Wann gibt es bei den Polargebieten eine totale Sonnenfinsternis?"
retrieved_docs = vector_retriever.invoke(query)


qa_chain_embeddings = (
    {"context": vector_retriever | RunnableLambda(format_docs), "question": RunnablePassthrough()}
    | prompt
    | llm
)

# Frage stellen
result = qa_chain_embeddings.invoke(query)
print(f"Antwort: {result.content}")
```

</details>